In [2]:
#Taller de Versionamiento y despliegue de modelos con pycaret y MLFLOW

In [3]:
#vamos a instalar las librerias necesarias
#pip install mlflow  pycaret fastapi uvicorn pydantic docker

In [ ]:
#Caso de estudio: Diabates
#Una clinica está buscando automatizar la predicción de diabetes en pacientes a partir de determinados indicadores medicos. Para cumplirlo se va a necesitar:
#1. Entrenar multiples modelos de ML y versionarlos con MLFLOW
#2. Crear un API que permita realizar la predicción
#3. Desplegar el API en un servidor on premise: Docker / Nginx


In [1]:
#Paso 1: Configurar el MlFLOW y cargar los datos
import mlflow
import pandas as pd
from pycaret.classification import *

#Configurar el MLFLOW:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
#localhost

#cargar el dataset:
data = pd.read_csv("diabetes_dataset.csv")

In [2]:
#Paso 2: Entrenar y registrar el modelo con MLFLOW
#Configurar el entorno con pycaret
exp = setup(data,target="Outcome",log_experiment=True,experiment_name="MLflow_Tracking",session_id=123,data_split_shuffle= True)

#Entrenar multiples modelos
best_model= compare_models(n_select=3) #compara los modelos que obtienes y traeme los tres primeros mejores.

#Guardar los modelos entrenados:
i=0
for model in best_model:
    save_model(model,f"best_model_{i}")
    i=i+1


,Description,Value
0,Session id,123
1,Target,Outcome
2,Target type,Binary
3,Original data shape,"(1000, 9)"
4,Transformed data shape,"(1000, 9)"
5,Transformed train set shape,"(700, 9)"
6,Transformed test set shape,"(300, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


AttributeError: 'ThreadLocalVariable' object has no attribute 'copy'

In [ ]:
#Creación de API Web usando FAST API
#Paso 1: definir la API: 

from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import pandas as pd

class ModelInput(BaseModel):
    Pregnancies: int
    Glucose:float
    BloodPressure:float
    SkinThickness:float
    Insulin:float
    BMI:float
    DiabetesPedigreeFunction:float
    Age:int

app= FastAPI()

model = pickle.load(open("best_model_0.pkl","rb"))

@app.post("/predict") #se usa para definir que una función recibirá solicitudes POST, es decir de un API
def predict(data: ModelInput):
    df= pd.DataFrame([data.dict()])
    prediction = model.predict(df)
    return {"prediction" : int(prediction[0])}




In [ ]:
#Despliegue del Modelo
#1. Creación de un docker:
#```dockerfile
#FROM python:3.9
#WORKDIR /app
# COPY /app
# RUN pip install fastapi uvicorn pydantic pickle-mixin mlflow
# CMD["uvicorn","main-app","--host","0.0.0.0","--port","8000"]

#construcción y ejecución de contenedor:
# docker build -t diabetes_api
# docker run -p 8000:8000 diabetes_api 
 